In [1]:
import os
import re

# orginal Image Rename

In [ ]:
import os

# ================================================ Folder Path =======================================================

original_folder = r"Original_Folder_link"
# ================================================ Orginal Image =======================================================

# Step 1: Rename original images
original_files = [f for f in os.listdir(original_folder) if os.path.isfile(os.path.join(original_folder, f))]
original_files.sort()

for i, file in enumerate(original_files, start=1):
    ext = file.split('.')[-1]
    old_path = os.path.join(original_folder, file)
    new_name = f"Image_{i}_original.{ext}"
    new_path = os.path.join(original_folder, new_name)
    os.rename(old_path, new_path)

print("Original images renamed.")


Original images renamed.


In [ ]:
import os
import re

# ================================================ Folder Path =======================================================
original_folder = r"Original_Folder_link"
noise_folder   = r"Noise_Image_Folder"

# ================================================ Helpers =======================================================
def list_images(folder):
    return sorted([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif'))])

def already_renamed(filename, suffix):
    # matches Image_<num>_<suffix>.<ext>
    return re.match(r'^Image_\d+_' + re.escape(suffix) + r'\.[A-Za-z0-9]+$', filename) is not None

def get_index_from_original(filename):
    # If original is already Image_<i>_original.ext -> get i
    m = re.match(r'^Image_(\d+)_original\.', filename)
    if m:
        return int(m.group(1))
    # Otherwise try to extract a number from filename (fallback)
    nums = re.findall(r'\d+', filename)
    return int(nums[0]) if nums else None

# ================================================ Original files info =======================================================
original_files = list_images(original_folder)
print(f"Found {len(original_files)} original images (sample first 5): {original_files[:5]}")

# If originals are not yet renamed, you may rename them here (commented out if already done):
# for i, file in enumerate(original_files, start=1):
#     ext = file.split('.')[-1]
#     old_path = os.path.join(original_folder, file)
#     new_name = f"Image_{i}_original.{ext}"
#     new_path = os.path.join(original_folder, new_name)
#     os.rename(old_path, new_path)
# print("Original images renamed.")

# After renaming check
original_files = list_images(original_folder)
print(f"After check, {len(original_files)} originals (sample): {original_files[:5]}")

# ================================================ Detect actual noise directory =======================================================
# The user may have noise files directly in noise_folder or inside a single subfolder.
candidates = [noise_folder] + [os.path.join(noise_folder, d) for d in os.listdir(noise_folder) if os.path.isdir(os.path.join(noise_folder, d))]
noise_dir = None
for c in candidates:
    imgs = list_images(c)
    if len(imgs) > 0:
        # choose the candidate with images - prefer top-level if it has images
        noise_dir = c
        break

if noise_dir is None:
    print("❌ No image files found in noise_folder or its immediate subfolders. Check your path.")
    raise SystemExit(1)

print(f"Using noise directory: {noise_dir}")
noise_files = list_images(noise_dir)
print(f"Found {len(noise_files)} noise images (sample first 5): {noise_files[:5]}")

# ================================================ Safe rename =======================================================
suffix = 'noise'
renamed_count = 0
skipped_count = 0

# If original filenames already Image_<i>_original.ext, map index -> filename to ensure correct pairing
orig_index_map = {}
for f in original_files:
    idx = get_index_from_original(f)
    if idx is not None:
        orig_index_map[idx] = f

# If originals are not in Image_i_original format, we'll just rely on sorted order.
for i, file in enumerate(noise_files, start=1):
    if already_renamed(file, suffix):
        skipped_count += 1
        continue

    ext = file.split('.')[-1]
    old_path = os.path.join(noise_dir, file)

    # Prefer using original index if available; otherwise use the loop index
    if i in orig_index_map:
        index_to_use = i
    else:
        # try to parse number from original filename at same position if possible
        index_to_use = i

    new_name = f"Image_{index_to_use}_{suffix}.{ext}"
    new_path = os.path.join(noise_dir, new_name)

    # If target exists already, skip (avoid overwrite)
    if os.path.exists(new_path):
        print(f"Skip rename, target exists: {new_path}")
        skipped_count += 1
        continue

    os.rename(old_path, new_path)
    renamed_count += 1

print(f"Renamed: {renamed_count}, Skipped (already correct or exists): {skipped_count}")

# ================================================ Final verification =======================================================
final_noise_files = list_images(noise_dir)
print(f"After renaming, {len(final_noise_files)} noise images (sample): {final_noise_files[:5]}")

# Quick pair-check (first 10)
pairs_ok = True
for i in range(1, min(11, len(original_files), len(final_noise_files)) + 1):
    orig_name = f"Image_{i}_original"
    noise_name = f"Image_{i}_{suffix}"
    o_exists = any(fname.startswith(orig_name) for fname in original_files)
    n_exists = any(fname.startswith(noise_name) for fname in final_noise_files)
    print(f"Pair {i}: original exists? {o_exists}, noise exists? {n_exists}")
    if not (o_exists and n_exists):
        pairs_ok = False

if pairs_ok:
    print("✅ Pairing looks OK for the checked samples.")
else:
    print("⚠️ Some pairs are missing — inspect filenames above for mismatches.")


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt

# ================================================ Folder Path =======================================================
original_folder = r"D:\E - Thesis Collection\DeepFrame-Reconstruction\B - Medial Image Dataset\Chest CT-Scan images Dataset\Original CT-Scan Image"
noise_folder = r"D:\E - Thesis Collection\DeepFrame-Reconstruction\B - Medial Image Dataset\Chest CT-Scan images Dataset\Noise Image"

# ================================================ Helper Function =======================================================
def find_image_path(folder, base_name):
    """Finds the image file with any extension (.jpg/.png/.jpeg)."""
    for ext in ['.jpg', '.png', '.jpeg']:
        path = os.path.join(folder, base_name + ext)
        if os.path.exists(path):
            return path
    return None

# ================================================ Load One Pair =======================================================
index = 5  # Change to test other pairs
original_base = f"Image_{index}_original"
noise_base = f"Image_{index}_noise"

original_path = find_image_path(original_folder, original_base)
noise_path = find_image_path(noise_folder, noise_base)

print("Original path:", original_path)
print("Noise path:", noise_path)

# Check if both files exist
if original_path is None or noise_path is None:
    print("❌ Error: Could not find one or both images. Please check file names or extensions.")
else:
    # Read images
    original_img = cv2.imread(original_path)
    noise_img = cv2.imread(noise_path)

    # Convert BGR → RGB for matplotlib display
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
    noise_img = cv2.cvtColor(noise_img, cv2.COLOR_BGR2RGB)

    # ================================================ Plot Side-by-Side =======================================================
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(noise_img)
    plt.title("Noise Image")
    plt.axis("off")

    plt.show()
